In [2]:
from summarization_llm.data import SumDataset
import datasets

In [3]:
data_module = SumDataset()
data_module.prepare_data()
data_module.setup("fit")
loader = data_module.train_dataloader()

In [18]:
from transformers import MT5ForConditionalGeneration
model_mt5 = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

In [4]:
import os
from pathlib import Path
import datasets
from summarization_llm.model import LlamaModule
import transformers
import unittest

model_name = 'meta-llama/Llama-2-13b-chat-hf'
model_config = transformers.AutoConfig.from_pretrained(model_name)
model = LlamaModule(model_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 6,553,600 || all params: 13,022,417,920 || trainable%: 0.05032552357220002


In [5]:
!nvidia-smi

Tue Sep  5 14:43:19 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB           On | 00000000:01:00.0 Off |                    0 |
| N/A   27C    P0               55W / 400W|   7777MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [6]:
for b in loader:
    b
    break

In [7]:
new_b = {k : b[k][0:1] for k in b}

In [10]:
new_b["labels"] = new_b["reference_ids"]

In [16]:
# del new_b["summary"]
# del new_b["id"]
del new_b["reference_mask"]


In [7]:
for k,v in new_b.items():
    print(v.size() if hasattr(v, "size") else len(v))

1
torch.Size([1, 128])
torch.Size([1, 128])
torch.Size([1, 73])
torch.Size([1, 73])


In [17]:
out = model.model(**new_b)

ValueError: Expected input batch_size (127) to match target batch_size (72).

In [48]:
new_b["labels"].size()[-1]

73

In [10]:
import torch
new_b["reference_ids"] = torch.nn.functional.pad(new_b["reference_ids"], (0, 128-new_b["reference_ids"].size()[-1]))

In [64]:
new_b["reference_ids"].size()

torch.Size([1, 128])

In [35]:
out.logits.size()

torch.Size([1, 128, 32000])

In [14]:
import torch.nn as nn

In [18]:
model_out = model(**new_b)

In [57]:
new_b["labels"] = new_b["reference_ids"]

In [59]:
del new_b["document"]

KeyError: 'document'

In [64]:
del new_b["reference_mask"]

In [68]:
temp = new_b["labels"]


In [70]:
new_b["labels"] = new_b["labels"][:, :12]

In [19]:
model_mt5_out = model_mt5(**new_b)


In [20]:
logits = model_mt5_out.logits

In [22]:
logits.size()

torch.Size([1, 73, 250112])

In [29]:

model_trunc_out = model_out.logits[:, :64, :]


In [39]:
soft = nn.Softmax(dim = 2)
import torch

In [21]:

from torch.nn import CrossEntropyLoss
loss_fct = CrossEntropyLoss(reduction="mean", ignore_index=-100)
loss_fct(logits.view(-1, logits.size(-1)), new_b["labels"].view(-1))

tensor(28.6377, grad_fn=<NllLossBackward0>)

In [49]:
model_trunc_out.view(-1, model_trunc_out.size(-1)).size()

torch.Size([64, 32000])

In [52]:
loss_fct = nn.CrossEntropyLoss()
loss_fct( model_trunc_out.view(-1, model_trunc_out.size(-1)) , new_b["reference_ids"].view(-1))



ValueError: Expected input batch_size (64) to match target batch_size (128).

In [67]:
out

tensor(18.4019, grad_fn=<ToCopyBackward0>)

In [10]:
for b in loader:
    out = model.training_step(b, 0)
    break

ValueError: Expected input batch_size (254) to match target batch_size (144).

In [8]:
b

{'document': ['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreci

In [9]:
for k,v in b.items():
    print(v.size() if hasattr(v, "size") else len(v))

2
2
2
torch.Size([2, 128])
torch.Size([2, 128])
torch.Size([2, 73])
torch.Size([2, 73])
torch.Size([2, 73])


In [34]:
8 * (4 * (4067 * 2 + 73 * 3))

267296

In [19]:
import torch

In [26]:
torch.cuda

<module 'torch.cuda' from '/home/semindan/.conda/envs/env_mamba/envs/llm_env/lib/python3.11/site-packages/torch/cuda/__init__.py'>

In [3]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Sep  5 13:07:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA 

In [14]:
!nvidia-smi

Tue Sep  5 12:52:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB           On | 00000000:C1:00.0 Off |                    0 |
| N/A   26C    P0               55W / 400W|   7777MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [5]:
d = data_module.setup("fit")

In [10]:
data_module.train[0].keys()

dict_keys(['document', 'summary', 'id', 'input_ids', 'attention_mask', 'reference_ids', 'reference_mask'])

In [17]:
len(data_module.train[2]["input_ids"])

4067

In [1]:
from summarization_llm.model import LlamaModule

In [1]:
from summarization_llm.tests.model_tests import TestModel
import unittest

In [2]:

a = TestModel()
a.test_model_init()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 6,553,600 || all params: 13,022,417,920 || trainable%: 0.05032552357220002


In [3]:
!nvidia-smi

Tue Sep  5 12:48:41 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB           On | 00000000:C1:00.0 Off |                    0 |
| N/A   26C    P0               55W / 400W|   7777MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [1]:
from summarization_llm.tests.pipeline_tests import TestPipeline
import unittest

pipe = TestPipeline()
pipe.test_one_train_step()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 6,553,600 || all params: 13,022,417,920 || trainable%: 0.05032552357220002


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.46 GiB (GPU 0; 39.43 GiB total capacity; 38.05 GiB already allocated; 389.75 MiB free; 38.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF